### https://towardsdatascience.com/how-to-forecast-sales-with-python-using-sarima-model-ba600992fa7d

In [1]:
import warnings
import itertools
import numpy as np
from numpy import concatenate, savetxt, unique, array, subtract
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
from pandas import merge, DataFrame, Series
import statsmodels.api as sm
import matplotlib
from math import sqrt
from statistics import mean
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'g'


In [2]:
df = pd.read_csv('CharlestownAllWithDate3.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
y = df.set_index(['Date'])


In [3]:
meter_ids = unique(y['MeterNo']) 
df2 = pd.DataFrame(index =['Date'], columns=['prediction', 'actual', 'error','mse','rmse'])
pred_date = '2021-02-01'

In [4]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 7)
SARIMAX: (0, 0, 1) x (0, 1, 0, 7)
SARIMAX: (0, 1, 0) x (0, 1, 1, 7)
SARIMAX: (0, 1, 0) x (1, 0, 0, 7)


In [5]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False, period=1)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:585: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it is not'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:585: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignor

In [6]:
for m in meter_ids:
    df = pd.read_csv('CharlestownAllWithDate3.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    y = df.set_index(['Date'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[1]], axis = 1, inplace = True)
    #print(y)
    mod = sm.tsa.statespace.SARIMAX(y,
                                order=(0, 1, 1),
                                seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    y_forecasted = pred.predicted_mean
    y_forecasted =y_forecasted.to_frame()
    y_truth = y[pred_date:]
    #mse = ((y_forecasted - y_truth) ** 2)
    df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
    df1.columns =['prediction', 'actual']
    df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
    df1['mse'] = df1['error'] ** 2
    df1['rmse'] = round(np.sqrt(df1['mse']),2)
    print(df1)
    df2 = df2.append(df1)

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


            prediction  actual  error     mse  rmse
Date                                               
2021-02-01    8.032382    10.0  -1.97  3.8809  1.97
2021-02-02    9.796398    11.0  -1.20  1.4400  1.20
2021-02-03    9.311478    10.0  -0.69  0.4761  0.69
2021-02-04    9.482739     9.0   0.48  0.2304  0.48
2021-02-05   10.299489     9.0   1.30  1.6900  1.30
...                ...     ...    ...     ...   ...
2021-04-07    5.288605     3.0   2.29  5.2441  2.29
2021-04-08    2.890617     3.0  -0.11  0.0121  0.11
2021-04-09    4.063380     2.0   2.06  4.2436  2.06
2021-04-10    1.997381     2.0  -0.00  0.0000  0.00
2021-04-11    2.951762     2.0   0.95  0.9025  0.95

[70 rows x 5 columns]


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


            prediction  actual  error       mse   rmse
Date                                                  
2021-02-01   35.712351    42.0  -6.29   39.5641   6.29
2021-02-02   36.657785    45.0  -8.34   69.5556   8.34
2021-02-03   40.625436    23.0  17.63  310.8169  17.63
2021-02-04   34.230814    27.0   7.23   52.2729   7.23
2021-02-05   30.838518    29.0   1.84    3.3856   1.84
...                ...     ...    ...       ...    ...
2021-04-07   11.991024    20.0  -8.01   64.1601   8.01
2021-04-08   15.188511    30.0 -14.81  219.3361  14.81
2021-04-09   20.529004    25.0  -4.47   19.9809   4.47
2021-04-10   21.455440     9.0  12.46  155.2516  12.46
2021-04-11   17.296790    12.0   5.30   28.0900   5.30

[70 rows x 5 columns]


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


...

            prediction  actual  error       mse   rmse
Date                                                  
2021-02-01   27.426281    43.0 -15.57  242.4249  15.57
2021-02-02   28.968509    19.0   9.97   99.4009   9.97
2021-02-03   24.986436    26.0  -1.01    1.0201   1.01
2021-02-04   25.317342    22.0   3.32   11.0224   3.32
2021-02-05   26.871711    24.0   2.87    8.2369   2.87
...                ...     ...    ...       ...    ...
2021-04-07   15.832529    11.0   4.83   23.3289   4.83
2021-04-08   12.746538    28.0 -15.25  232.5625  15.25
2021-04-09   19.685456    12.0   7.69   59.1361   7.69
2021-04-10   17.750955    11.0   6.75   45.5625   6.75
2021-04-11   14.797820    12.0   2.80    7.8400   2.80

[70 rows x 5 columns]


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


            prediction  actual  error       mse   rmse
Date                                                  
2021-02-01   26.677351    30.0  -3.32   11.0224   3.32
2021-02-02   28.230647    31.0  -2.77    7.6729   2.77
2021-02-03   27.723611    17.0  10.72  114.9184  10.72
2021-02-04   26.170387    13.0  13.17  173.4489  13.17
2021-02-05   22.327175     7.0  15.33  235.0089  15.33
...                ...     ...    ...       ...    ...
2021-04-07    6.794973     7.0  -0.21    0.0441   0.21
2021-04-08    7.482014     6.0   1.48    2.1904   1.48
2021-04-09    5.805076    21.0 -15.19  230.7361  15.19
2021-04-10   10.157674     3.0   7.16   51.2656   7.16
2021-04-11    8.415234    10.0  -1.58    2.4964   1.58

[70 rows x 5 columns]


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


            prediction  actual  error       mse   rmse
Date                                                  
2021-02-01   75.370418    82.0  -6.63   43.9569   6.63
2021-02-02   78.814770    74.0   4.81   23.1361   4.81
2021-02-03   75.430915    70.0   5.43   29.4849   5.43
2021-02-04   72.412270    59.0  13.41  179.8281  13.41
2021-02-05   64.578927    65.0  -0.42    0.1764   0.42
...                ...     ...    ...       ...    ...
2021-04-07   12.113587    25.0 -12.89  166.1521  12.89
2021-04-08   21.675024    22.0  -0.32    0.1024   0.32
2021-04-09   21.013048    23.0  -1.99    3.9601   1.99
2021-04-10   21.694208    21.0   0.69    0.4761   0.69
2021-04-11   24.028383    23.0   1.03    1.0609   1.03

[70 rows x 5 columns]


In [7]:
def Summary(x):
    return Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()])
df2.apply(Summary)

,prediction,actual,error,mse,rmse
min,-2.811029,0.000000,-70.980000,0.000000,0.000000
max,97.907475,103.000000,50.670000,5038.160400,70.980000
mean,16.618180,16.314338,0.303825,42.446427,4.332457
sum,319899.968550,314051.000000,5848.640000,817093.714200,83399.800000
count,19250.000000,19250.000000,19250.000000,19250.000000,19250.000000


In [8]:
df2.drop(df2.columns[[2,3,4]], axis = 1, inplace = True)


In [9]:
df3 = df2.groupby(df2.index)['prediction','actual'].sum()

In [10]:
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['mse'] = df3['error'] ** 2
df3['rmse'] = round(np.sqrt(df3['mse']),2)
df3['%error'] = (df3['error']/df3['actual'])*100
print(df3)
df3.to_csv('CHL_SARIMA_AllMeters_ByDate_NoGaps.csv')

                      prediction  actual    error           mse     rmse  \
2021-02-01 00:00:00  5660.232267  6196.0  -535.77  2.870495e+05   535.77   
2021-02-02 00:00:00  5859.005286  5635.0   224.01  5.018048e+04   224.01   
2021-02-03 00:00:00  5662.954494  5163.0   499.95  2.499500e+05   499.95   
2021-02-04 00:00:00  5432.666491  4432.0  1000.67  1.001340e+06  1000.67   
2021-02-05 00:00:00  5069.145552  4379.0   690.15  4.763070e+05   690.15   
...                          ...     ...      ...           ...      ...   
2021-04-08 00:00:00  3543.596281  4075.0  -531.40  2.823860e+05   531.40   
2021-04-09 00:00:00  3835.002882  3925.0   -90.00  8.100000e+03    90.00   
2021-04-10 00:00:00  3880.738111  3719.0   161.74  2.615983e+04   161.74   
2021-04-11 00:00:00  3957.628724  4044.0   -86.37  7.459777e+03    86.37   
Date                    0.000000     0.0     0.00  0.000000e+00     0.00   

                        %error  
2021-02-01 00:00:00  -8.647030  
2021-02-02 00:00:00  

In [11]:
df3.apply(Summary)


,prediction,actual,error,mse,rmse,%error
min,0.000000,0.000000,-1516.830000,0.000000e+00,0.000000,-22.919764
max,8207.525681,8628.000000,1968.160000,3.873654e+06,1968.160000,37.531655
mean,4505.633360,4423.253521,82.380423,3.421994e+05,433.621268,2.941582
sum,319899.968550,314051.000000,5849.010000,2.429616e+07,30787.110000,205.910728
count,71.000000,71.000000,71.000000,7.100000e+01,71.000000,70.000000


In [12]:
df4 = df3.mean()
df4

prediction      4505.633360
actual          4423.253521
error             82.380423
mse           342199.434565
rmse             433.621268
%error             2.941582
dtype: float64